In [ ]:
import os
from tabula import read_pdf
import pandas as pd

def find_folder_path_recursive(main_path, target_folder):
    try:
        for item_name in os.listdir(main_path):
            item_path = os.path.join(main_path, item_name)

            if os.path.isdir(item_path):
                if item_name == target_folder:
                    return item_path  # Folder found, return its path
                else:
                    # Recursively search in subdirectories
                    result = find_folder_path_recursive(item_path, target_folder)
                    if result:
                        return result  # Folder found in a subdirectory, return its path

    except Exception as e:
        print(f"Error searching for folder path: {e}")

    return None  # Folder not found

def process_pdf_file(file_path):
    try:
        # Use tabula to extract tables from the PDF
        tables = read_pdf(file_path, pages='all', multiple_tables=True)

        # If no tables are found
        if not tables:
            print(f"No tables found in the PDF: {file_path}")
            return None

        return tables  # Return all tables

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def process_files_in_folder(folder_path):
    total_amounts = []  # List to store total amounts

    try:
        for item_name in os.listdir(folder_path):
            item_path = os.path.join(folder_path, item_name)
            if item_name.endswith(".pdf"):
                tables = process_pdf_file(item_path)
                if tables:
                    table = tables[0].set_index('TRRN No :')
                    amount_str = table.T['Total Amount (Rs) :'].iloc[0];
                    amount = int(amount_str.replace(',', ''))
                    total_amounts.append(amount);
                

    except Exception as e:
        print(f"Error processing files in folder: {e}")

    return total_amounts

def find_and_process_total_amounts(main_path, folder_name):
    # Find full path of the specified folder
    folder_path = find_folder_path_recursive(main_path, folder_name)
    if folder_path:
        # Get total amounts for all PDFs in the specified folder
        total_amounts_array = process_files_in_folder(folder_path)

        return total_amounts_array
    else:
        print(f"Folder not found: {folder_name}")
        return None

# Example usage
main_path = "/home/finstein-emp/Desktop/Post Disbursement/Sep"
folder_name = "PF paid challan Sep 23"

# Get total amounts for all PDFs in the specified folder
total_amounts_array = find_and_process_total_amounts(main_path, folder_name)

# Display the array of total amounts
print("Array of Total Amounts:", total_amounts_array)
